In [104]:
import requests
import random
import numpy as np

TOKEN = "pk_18e2ba4b80cb471aa96ba58fadec6b05" # edit if using your own IEX account
SYMBOLS = """abde,amd,googl,goog,adi,aapl,amat,asml,adsk,bidu,
    avgo,cdns,cern,chkp,csco,ctxs,fb,intc,intu,klac,
    lrcx,mxim,mchp,mu,msft,ntap,ntes,nvda,nxpi,qcom,
    swks,symc,snps,txn,vrsn,wdc,wday,xlnx"""
TYPES = "quote"

query = {"token": TOKEN, "symbols": SYMBOLS, "types": TYPES}
r = requests.get("https://cloud.iexapis.com/beta/stock/market/batch", params=query)
dict = r.json()
    
estimatedPrices = {}
for symbol in dict:
    estimatedPrices[symbol] = random.random() * dict[symbol]["quote"]["latestPrice"]
    + random.random() * dict[symbol]["quote"]["latestVolume"]
    + random.random() * dict[symbol]["quote"]["marketCap"]
    + random.random() * dict[symbol]["quote"]["week52High"]
    + random.random() * dict[symbol]["quote"]["week52Low"]
    + random.random() * dict[symbol]["quote"]["peRatio"]
    
arr = np.array([0, 0, 0, 0, 0, 0, 0]) # python is weird

for symbol in dict:
    latestPrice = dict[symbol]["quote"]["latestPrice"]
    latestVolume = dict[symbol]["quote"]["latestVolume"]
    marketCap = dict[symbol]["quote"]["marketCap"]
    week52High = dict[symbol]["quote"]["week52High"]
    week52Low = dict[symbol]["quote"]["week52Low"]
    peRatio = dict[symbol]["quote"]["peRatio"]
    
    arr = np.vstack([arr, [latestPrice, latestVolume, marketCap, week52High, week52Low, peRatio, estimatedPrices[symbol]]])
    
arr = arr[1::]
print(arr)


[[ 2.78900000e+01  1.27884912e+08  2.80378170e+10  3.41400000e+01
   9.04000000e+00  8.20300000e+01  1.29675052e+00]
 [ 1.23613000e+03  1.56801400e+06  8.59465119e+11  1.29144000e+03
   9.77660000e+02  2.79600000e+01  4.70542039e+02]
 [ 1.23154000e+03  1.19586600e+06  8.56273752e+11  1.27389000e+03
   9.70110000e+02  2.78600000e+01  2.20180108e+02]
 [ 1.10180000e+02  3.71685600e+06  4.05808365e+10  1.11120000e+02
   7.66200000e+01  2.59200000e+01  5.69674234e+01]
 [ 1.95090000e+02  5.09372760e+07  9.19903975e+11  2.33470000e+02
   1.42000000e+02  1.59000000e+01  1.38550938e+02]
 [ 4.13300000e+01  1.31144080e+07  3.92384127e+10  6.15700000e+01
   2.87900000e+01  1.03600000e+01  2.38969637e+01]
 [ 1.94360000e+02  7.67474000e+05  8.07159514e+10  2.21660000e+02
   1.44500000e+02  2.77300000e+01  1.30574595e+02]
 [ 1.58820000e+02  1.55002100e+06  3.47830094e+10  1.69050000e+02
   1.17720000e+02 -4.23750000e+02  2.45929387e+01]
 [ 1.71770000e+02  2.08941700e+06  4.76381628e+09  2.84220000e+0

In [47]:
print(estimatedPrices)
    


{'AMD': 15.36383570111675, 'GOOGL': 574.6344093428089, 'GOOG': 1140.2834840502762, 'ADI': 90.00803647344009, 'AAPL': 125.3712047408982, 'AMAT': 12.433946065907005, 'ASML': 128.43396505833908, 'ADSK': 21.30280320605947, 'BIDU': 93.0444963706757, 'AVGO': 153.08402147065905, 'CDNS': 14.640765277769603, 'CERN': 7.218017614417411, 'CHKP': 50.17924575610996, 'CSCO': 22.443317590434752, 'CTXS': 66.86943191064027, 'FB': 47.5344054684495, 'INTC': 3.020682410644192, 'INTU': 203.71229603579403, 'KLAC': 92.21918235189852, 'LRCX': 15.157874787300816, 'MXIM': 5.041917297038374, 'MCHP': 83.5669981138582, 'MU': 29.717135662276533, 'MSFT': 85.89887814166597, 'NTAP': 15.709797308010508, 'NTES': 42.461278297666816, 'NVDA': 129.2933396591308, 'NXPI': 86.34506070188006, 'QCOM': 33.31597650984758, 'SWKS': 69.21216742327316, 'SYMC': 3.4748186690248284, 'SNPS': 109.54812549361557, 'TXN': 51.583938226569735, 'VRSN': 22.168020302510882, 'WDC': 24.58174128028005, 'WDAY': 64.370323099042, 'XLNX': 39.6532969131626